# TFRecord  : 만들었던 함수들이 무슨역할을 하는지 흐름을 숙지해야함.. 

## 순서 .. 
 1. 하나의 데이터를 구성할 feature들을 모두 tf.train.Feature 로 변환(인코딩)한다. --> feature type으로.. 
 2. 인코딩 된 기능 값으로 딕셔너리를 생성한다.-> tf.train.Example(feature = tf.train.Feature(딕셔너리)) 처리 해준다... 이그잼플이 인식하게..
 3. Example을 TFRecord 에 저장하기 위한 형태인 bytes로 변환 -> SerializeToString()이용
 
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- 데이터 양이 많을때 시간이 많이 걸리니까 사용하는 것 이다. 
- Tensorflow에서 제공하는 데이터셋을 파일로 저장하는방식. -> 어떤 데이터를 파일로 저장하는것을 직렬화한다고 생각하면 된다 / 한줄로 쭉펼쳐서 저장 
    - 파일에서 메모리로 갖고오는게 시간이 많이걸림.. 
    - 데이터 양이 많을 경우 이를 Binary로 Seralization(직렬화)하여 하나의 파일로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer(파일포멧) 형태로 Serialization을 수행해서 저장할 수 있는 TFRecords 파일 포맷 형태를 지원한다. 
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecords 파일에 저장할 수 있다.
- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.
- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, byte 타입을 변환 -> 이미지파일.. 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환 -> boolean, 정수 변환..
- tf.tran.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```

In [1]:
import tensorflow as tf
type(tf.constant(0))

tensorflow.python.framework.ops.EagerTensor

In [2]:
# tf.train.Feature 객체들을 생성하는 (기본차입의 값들을 Featrue로 변환하는) 함수 구현
# 고정적으로 사용되는 함수 이다.. 로직에 따라서 바뀌거나 하지 않는.. 


def _bytes_feature(value):
    """
    value로 strin, byte를 받아서 BytesList로 변환하는 함수
    """
    # value가 Tensor(텐서플로우의 배열 타입)타입인 경우 ndarray로 변환 -> binary ㅍ일 
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  
    return tf.train.Feature(bytes_list = tf.train.BytesList(value =[value]))

def _float_feature(value):
    """
    float타입의 value를 받아서 FloatList 변환 하는 함수 

    """
    return tf.train.Feature(float_list = tf.train.FloatList(value =[value]))

def _int64_feature(value):
    """
    int,uint,bool, 타입의 value를 받아서 Int64List로 변환하는 함수
    """
    return tf.train.Feature(int64_list= tf.train.Int64List(value = [value]))

In [4]:
v = _int64_feature(20)
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
int64_list {
  value: 20
}



In [5]:
v = _float_feature(56.7)
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
float_list {
  value: 56.7
}



In [6]:
v = _int64_feature(True)  # 불리언값 -> True =1, False = 0 으로 변환..
print(v)

int64_list {
  value: 1
}



In [9]:
v =_bytes_feature(b'hello') # 문자열, byte -> 둘다 byte타입을 전달.. // 문자열 입력할때 앞에 b 를 붙여서 바이트 타입으로 전달 
print(v)

bytes_list {
  value: "hello"
}



In [10]:
s = '홍길동' # 일반 string일 경우 -> byte타입으로 변환한 뒤에  BytesList로 변환

In [12]:
_bytes_feature(s.encode('utf-8'))  # 한글 같은 경우 인코딩해서 나온형태가 된다 .. 영어는 그대로 나오지만.. 

bytes_list {
  value: "\355\231\215\352\270\270\353\217\231"
}

In [13]:
# binary 파일 (이미지) //--> 파이썬에서 바이너리는 바이츠 타입으로 말한다.. 
with open('lion.jpg', 'rb') as f:
    img = f.read()
    
type(img)

bytes

In [14]:
v = _bytes_feature(img)
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\001\000`\000`\000\000\377\376\000<CREATOR: gd-jpeg v1.0 (using IJG JPEG v62), quality = 100\n\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\333\000C\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\300\000\021\010\001\302\002\200\003\001\021\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\00

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력(바깥으로 내보내서 tfrecord파일로 저장하기 전에 변환해야 한다.

In [15]:
feature = _float_feature(30.2)
print(feature)

float_list {
  value: 30.2
}



In [16]:
v = feature.SerializeToString() # 출력형태로 변환. -> String 을  bytes 라고 생각하면됨.. 
type(v)

bytes

In [17]:
print(v)

b'\x12\x06\n\x04\x9a\x99\xf1A'


# TFRecord 저장 예제

### tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.


### TFRecord로 저장할 toy dataset을 생성

In [47]:
# 가상의 데이터 만들기.. // 만들어서 TFRecord 연습
import numpy as np
N_DATA = 1000 #dataset의 데이터 개수.

#boolean 1000개 생성
feature0 = np.random.choice([False, True], N_DATA)
#feature0.shape
#정수 1000개
feature1 = np.random.randint(0,5,N_DATA)
# string 1000개
str_list = np.array([b'lion',b'tiger',b'cat',b'dog',b'bear'])
feature2 = str_list[feature1]
# float 1000
feature3 = np.random.randn(N_DATA) # randn 정규분포숫자...
feature0.shape,feature1.shape, feature2.shape, feature3.shape

((1000,), (1000,), (1000,), (1000,))

In [49]:
def serialize_example(feature0, feature1,feature2, feature3):
    """
    하나의 데이터의 속성(Feature)값들을 받아서 Exaple을 생성한 뒤 그 Example을 출력 가능한 Bytes로 만들어 반환 (SerializeToString()이용)
    feature 0: bool, feature1: int, feature2: bytes(string), feature3:float
    """
    # feature들을 dictionary로 생성
    feature ={
        'feature0': _int64_feature(feature0),
        'feature1': _int64_feature(feature1),
        'feature2': _bytes_feature(feature2),
        'feature3': _float_feature(feature3),  # value 값은 다 feature 값이어야 하마..
    }
    # feature들을 가진 tf.train.Example 객
    체를 생성
    example = tf.train.Example(features = tf.train.Features(feature = feature)) # 이그잼플이 인식할수있게 딕셔너리를 피쳐화해서 넣어줌
    print(type(example))
    
    # Example을 TFRecord 에 저장하기 위한 형태인 bytes(=binary string )로 변환 -> SerializeToString()이용
    return example.SerializeToString()

### 출력 처리
- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.-> TFRecord 파일로 저장한다.

In [25]:
import os
# tfrecord 파일 저장할 디렉토리 
if not os.path.isdir('tfrecord'):
    os.mkdir('tfrecord')

In [31]:
# TFRecordWriter 객체를 생성 -> TFRecord파일로 직렬화된 Example을 출력하는 메소드 제공.
tfrecord_file_path = './tfrecord/data.tfr' # 확장자 : tfr, record, tfrecord  -> 셋중에 하나를 사용한다.
tfrecord_writer = tf.io.TFRecordWriter(tfrecord_file_path)   # 객체 생성.. 

In [50]:
for data in zip(feature0, feature1, feature2, feature3):
    #print(data)
    sv = serialize_example(bool(data[0]), data[1], data[2],data[3]) # 반환된 직렬화된 feature를 갖고있는 example을 저장, 
    tfrecord_writer.write(sv)
    
tfrecord_writer.close() # 출력 Stream 연결 닫기.
# 하나씩 출력해서 data에 쌓아서 저장.. 

(False, 2, b'cat', -0.6646572538717315)
<class 'tensorflow.core.example.example_pb2.Example'>


TypeError: expected bytes, method found

# TFRecord파일 읽기 및 역직렬화(Deserialize)  
 - 읽기  = 역직렬화  -> 입력.. 
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [ ]:
#def serialize_example(feature0, feature1,feature2, feature3) : 함수를 역순으로 한다고 생각하면됨..

In [34]:
def _parse_function(tfrecord_serialized): # 4교시...
    """
    serialize(직렬화-bytes)된 Example 데이터 하나를 받아서 역직렬화 한뒤에 반환하는 함수
    [매개변수]
        tfrecord_serialized : 직렬화된 Example
    [반환값]
        Example구성 Feature 들. (featrue 0,1,2,3)
    """
    # 역직렬화 해서 읽어온 Feature들을 저장할 Feature들을 dictionary로 구성
    # 이름: 저장할때 사용한 name, value: 읽어온 Feature를 저장할 빈 Feature (역직렬화할 값의 타입을 선언한다.)
    feature ={
        'feature0': tf.io.FixedLenFeature([], tf.int64),
        'feature1': tf.io.FixedLenFeature([], tf.int64),
        'feature2': tf.io.FixedLenFeature([], tf.string),  # bytes.. 템서플로어에서  타입정하는 상수.. 
        'feature3': tf.io.FixedLenFeature([], tf.float32),
    }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, feature)
    # tf.cast(값, dtype) 값의 데이터 타입을 지정한 dtype으로 반환
    feature0 = tf.cast(parsed_features['feature0'], tf.bool)
    feature1 = tf.cast(parsed_features['feature1'], tf.int64)
    feature2 = tf.cast(parsed_features['feature2'], tf.string)
    feature3 = tf.cast(parsed_features['feature3'], tf.float32)
    return feature0, feature1, feature2, feature3  # 반환값 튜플 

### tfrecord 에 모든 학습해야할 내용이 들어있는 것이므로 로컬에 이미지가 없더라도 학습이 가능하다.~!

In [ ]:
#읽어올 데이터 셋이  data.tfr 파일로 하드디스크에 저장되어 있으니까 // 다른 함수를 통해서 읽어들이면 dataset이 되는 원리..
# dnn4 분류회기 에설명 map, filter

In [35]:
# tfrecord 파일 읽기// data 모음.map , data모음.filter / map 함수 처리 결과를 반환/ filter는 불값으로 데이터 걸러냄
dataset = tf.data.TFRecordDataset(tfrecord_file_path).map(_parse_function) # 경로 넣어줌.../ 함수 넣어서 리턴값을 넣어줌..// 반환값 튜플

In [43]:
# dataset에서 3개의 데이터만 조회
for data in dataset.take(3):
    #print(type(data))  
    #print(data)
    print(data[0].numpy(), data[1].numpy(),data[2].numpy(), data[3].numpy())

True 4 b'bear' -1.1441451
False 3 b'dog' 0.3271088
False 0 b'lion' -0.96352994


In [ ]:
# tensor 
(<tf.Tensor: shape=(), dtype=bool, numpy=True>, <tf.Tensor: shape=(), dtype=int64, numpy=4>, <tf.Tensor: shape=(), dtype=string, numpy=b'bear'>, <tf.Tensor: shape=(), dtype=float32, numpy=-1.1441451>)

In [45]:
feature0[0],feature1[0],feature2[0], feature3[0]

(True, 4, b'bear', -1.1441451790473685)